In [12]:
import os
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# ✅ Ensure model directory exists
os.makedirs("model", exist_ok=True)

# Load dataset
df = pd.read_csv("/root/heart.csv")

df.replace("?", pd.NA, inplace=True)
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")

df["target"] = (df["num"] > 0).astype(int)

X = df.drop(columns=["id", "dataset", "num", "target"], errors="ignore")
y = df["target"]

X = X.select_dtypes(include=["int64", "float64"])

imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X)

X_train, _, y_train, _ = train_test_split(
    X_imputed, y, test_size=0.25, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# ✅ Now these WILL work
joblib.dump(imputer, "model/imputer.pkl")
joblib.dump(scaler, "model/scaler.pkl")


# Models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost": XGBClassifier(
        eval_metric="logloss",
        random_state=42,
        use_label_encoder=False
    ),
}

# Train and save models
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    joblib.dump(model, f"model/{name.replace(' ', '_')}.pkl")

print("✅ All models trained and saved successfully.")


/usr/local/lib/python3.12/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['sex' 'cp' 'restecg' 'slope' 'thal']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


✅ All models trained and saved successfully.


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [02:35:50] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
